This is a project analyzing data from a Star Wars survey. The dataset is obtained from [here](https://github.com/fivethirtyeight/data/tree/master/star-wars-survey).

The data has several columns, including the ones below. There are several other columns containing answers to questions about the Star Wars movies. For some questions, the respondent had to check one or more boxes.

* `RespondentID` — An anonymized ID for the respondent (person taking the survey)
* `Gender` — the respondent's gender
* `Age` — the respondent's age
* `Household Income` — the respondent's income
* `Education` — the respondent's education level
* `Location (Census Region)` — the respondent's location
* `Have you seen any of the 6 films in the Star Wars franchise?` — a Yes or No response
* `Do you consider yourself to be a fan of the Star Wars film franchise?` — a Yes or No response

---
# Data cleaning

Let's load and explore the dataset.

In [2]:
import pandas as pd

survey = pd.read_csv("data/StarWars.csv", encoding="ISO-8859-1")

print(survey.info())
print(survey.columns)
print(survey.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187 entries, 0 to 1186
Data columns (total 38 columns):
 #   Column                                                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                                                         --------------  -----  
 0   RespondentID                                                                                                                                   1186 non-null   float64
 1   Have you seen any of the 6 films in the Star Wars franchise?                                                                                   1187 non-null   object 
 2   Do you consider yourself to be a fan of the Star Wars film franchise?                                                                          837 non-null    object 
 3   Which of the following Star 